# Summit Schools Manager of Data Performance Task
## Prepared by Justin August

Note: I tried to make as much of this code reusable as possible by creating functions that took appropriate arguments with defaults as defined by the tasks. With a few tweaks to the way constants and arguments are inputted this could be converted into a script that would run regularly from the command line via chron or manually as well as taking user input from the command line to specify sites and data sources.

In [1]:
import pandas as pd
from datetime import date

In [2]:
#Define today's date for filename usage later on.
TODAY = date.today().strftime('%m-%d-%Y')

### A dictionary to hold site metadata such as shortname and site name each site.

_This would need to be expanded and customized for each data source as schools were added_
_I could see information like this being kept in a 'sites_info.py' file or module to load into our notebooks so there was a uniform way of storing and accessing this information_

In [3]:
# Create Dictionary with site_id as keys containing short_name and site_name
CA_SITE_META = {2:
                {'short_name':'Tahoma',
                 'site_name' : 'Summit Public School: Tahoma'
                },
             3:
                {'short_name':'Prep',
                 'site_name' : 'Summit Preparatory Charter High School'
                },
             4:
                {'short_name':'Everest',
                 'site_name' : 'Everest Public High School'
                },
             5:
                {'short_name':'Denali',
                 'site_name' : 'Summit Public School: Denali'
                },
             6:
                {'short_name':'Shasta',
                 'site_name' : 'Public School: Shasta'
                },
             7:
                {'short_name':'K2',
                 'site_name' : 'Summit Public School: K2 Summit'
                },
             8:
                {'short_name':'Tamalpais',
                 'site_name' : 'Summit Public School: Tamalpais'
                },
            }

## Pull out site_ids for later use
ALL_CA_SITES = list(CA_SITE_META.keys())

## Part 1: School Rosters

In [4]:
# Fetch data for ELA Statuses and put it into a DataFrame
ELA_STATUSES_URL = 'https://drive.google.com/uc?export=download&id=1dK-050RcSingosBwLcuXBmrZavcfL8-j'
ela_statuses_df = pd.read_csv(ELA_STATUSES_URL)

### These are the data specified to be contained within the enrollment files

In [5]:
#define columns for enrollment filese
ROSTER_DATA_COLUMNS = ['LOCAL_STUDENT_ID',
                'STATE_STUDENT_ID',
                'SITE_ID',
                'SITE_NAME',
                'FIRST_NAME',
                'LAST_NAME',
                'GRADE_LEVEL',
                'CURRENT_SCHOOL_ENROLLMENT_START_DATE',
                'CURRENT_SCHOOL_ENROLLMENT_END_DATE'
               ]

### This function will be default run a roster report on all sites in CA using the given DataFrame.
Individual or subsets of sites can be run by placing the site IDs in a list. Additionally a separate DataFrame can be called.

In [6]:
def create_rosters(site_ids = ALL_CA_SITES,
                   df = ela_statuses_df
                  ):
    """
    This function will be default run a roster report on all sites in CA using the given DataFrame.

    Individual or subsets of sites can be run by placing the site IDs in a list. Additionally a separate DataFrame can be called.

    Arguments:
    - df - DataFrame containing data and metadata about students and their ELA proficiency (Default: ALL_CA_SITES)
    - site_ids - a list of numeric site ids to create testing lists for (Default: ela_statuses_df)
    
    Output:
    - Writes CSV files based on site_ids
    - Prints confirmation
    - Returns boolean True
    
    """
    

    for site_id in site_ids:
        
        # Get site's shortname to use in the filename
        site_shortname = CA_SITE_META[site_id]['short_name']


        # filename as specified
        filename = f'{site_shortname}_{site_id}_Roster_{TODAY}.csv'

        # Write files print confirmation
        df.loc[df['SITE_ID'] == site_id,ROSTER_DATA_COLUMNS].to_csv(filename)
            
        print(f'Roster for {site_shortname} written to {filename}')
    
    print('All sites data written.')
    return(True)

In [7]:
create_rosters()

Roster for Tahoma written to Tahoma_2_Roster_10-17-2020.csv
Roster for Prep written to Prep_3_Roster_10-17-2020.csv
Roster for Everest written to Everest_4_Roster_10-17-2020.csv
Roster for Denali written to Denali_5_Roster_10-17-2020.csv
Roster for Shasta written to Shasta_6_Roster_10-17-2020.csv
Roster for K2 written to K2_7_Roster_10-17-2020.csv
Roster for Tamalpais written to Tamalpais_8_Roster_10-17-2020.csv
All sites data written.


True

## Part 2: English Proficiency Testing Lists

In [8]:
# Data URL for mentors
MENTOR_DATA_URL = 'https://drive.google.com/uc?export=download&id=1wpKxw2rWB1a7jQBDunay0JfSO6lUiJ67'

# Pull in and merge both sheets from the remote Excel document
# This assumes the workbooks are consistent format with two sheets of data
# If data would be variable then this would need to be rewritten using ExcelFile class
# and some logic to sniff out appropriate sheets.
mentor_df = pd.merge(pd.read_excel(MENTOR_DATA_URL, sheet_name = 0),
                       pd.read_excel(MENTOR_DATA_URL, sheet_name = 1))

### Define Default ELA Statuses of Interest

In [9]:
ELA_STATUSES = ['EL','TBD']

### Define columns for final output
_Case and Spacing will be corrected before output_

In [10]:
# Existing columns in the DataFrames to be included
ELA_PROF_DATA_COLUMNS = ['LOCAL_STUDENT_ID',
                'STATE_STUDENT_ID',
                'SITE_NAME',
                'FIRST_NAME',
                'LAST_NAME',
                'GRADE_LEVEL',
                'MENTOR_FIRST_NAME',
                'MENTOR_LAST_NAME',
                'CURRENT_ELA_STATUS',
                'ELA_PRIMARY_LANGUAGE'
               ]

# Blank columns to be inserted into the dataframe.
ELA_PROF_BLANK_COLUMNS = ['Notification Letter Sent Home',
                 'Date Notification Letter Sent Home',
                 'Date Listening Completed',
                 'Date Reading Completed',
                 'Date Writing Completed',
                 'Date Speaking Completed',
                 'Date Assessment Completed',
                 'Assessment Deadline',
                 'Notes'
                ]

### This function will by default create a testing list in XLSX format with sheets for all sites in CA using the mentor DataFrame and ELA Status DF, filtering by ELA Statuses of "ELA" and "TBD".
- Individual or subsets of sites can be run by placing the site IDs in a list.
- Different ELA statuses could be defined as well via a list.

In [11]:
def create_test_lists(mentor_df = mentor_df,
                      ela_statuses_df = ela_statuses_df,
                      ela_statuses_ = ELA_STATUSES,
                      site_ids = ALL_CA_SITES
                     ):
    """
    This function will by default create a testing list in XLSX format with sheets for all sites in CA using the mentor DataFrame and ELA Status DF, filtering by ELA Statuses of "ELA" and "TBD".

    Individual or subsets of sites can be run by placing the site IDs in a list.
    Different ELA statuses could be defined as well via a list.
    
    Arguments:
    - mentor_df: DataFrame containing student IDs, mentor IDs and mentor metadata (Default: mentor_df)
    - ela_statuses_df - DataFrame containing data and metadata about students and their ELA proficiency (Default: ela_statuses_df)
    - ela_statuses_ - a list of ELA statuses to filter into the testing lists (Default: ELA_STATUSES)
    - site_ids - a list of numeric site ids to create testing lists for (Default: ALL_CA_SITES)
    
    Output:
    - Writes xslx file with sheets based on site_ids
    - Prints confirmation
    - Returns boolean True

    """
    
    
   # Correct column names to save time later on a subsequent merge
    mentor_df.columns = ['LOCAL_STUDENT_ID', 'MENTOR_GROUP_ID', 'MENTOR_ID', 'MENTOR_FIRST_NAME',
           'MENTOR_LAST_NAME', 'MENTOR_FULL_NAME']
    
    # Merge ELA Status data with mentor data using 'LOCAL_STUDENT_ID'
    test_list_data = pd.merge(ela_statuses_df,
                           mentor_df,
                           on = 'LOCAL_STUDENT_ID'
                          )
    
    # Filter the data down to relevant columns needed
    test_list_data = test_list_data.loc[test_list_data['CURRENT_ELA_STATUS'].isin(ela_statuses_),
                                  ELA_PROF_DATA_COLUMNS]
    
    #Append empty columns
    test_list_data[ELA_PROF_BLANK_COLUMNS] = ''

    #Fix columns case from import to match requirements
    fixed_columns = []
    for column in test_list_data.columns:
        fixed_columns.append(column.title().replace("_"," ").replace(" Id"," ID").replace('Ela ','ELA '))
    test_list_data.columns = fixed_columns

    
    
    test_list_filename = f'SPS_English_Proficiency_Testing_Lists_All_Schools_{TODAY}.xlsx'
    
    # Write file and print confirmation
    
    with pd.ExcelWriter(path = test_list_filename, mode='w') as writer:

        print(f'Writing data to {test_list_filename}')

        for site_id in site_ids:
            site_shortname = CA_SITE_META[site_id]['short_name']
            site_name = CA_SITE_META[site_id]['site_name']
            
            # Write data
            test_list_data.loc[test_list_data['Site Name'] == site_name].to_excel(writer,
                                                                        sheet_name = site_shortname)
            print(f'Data for {site_name} written.')
            
    print('All sites data written.')
    return(True)

In [12]:
create_test_lists()

Writing data to SPS_English_Proficiency_Testing_Lists_All_Schools_10-17-2020.xlsx
Data for Summit Public School: Tahoma written.
Data for Summit Preparatory Charter High School written.
Data for Everest Public High School written.
Data for Summit Public School: Denali written.
Data for Public School: Shasta written.
Data for Summit Public School: K2 Summit written.
Data for Summit Public School: Tamalpais written.
All sites data written.


True